In [30]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch
import time

datapath = '../data-unversions/p1ch7/'
cifar10 = datasets.CIFAR10(root= datapath, train=True, download = True, transform=transforms.ToTensor())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Files already downloaded and verified


Pre-processing

In [31]:
imgs = torch.stack([img_t for img_t, _ in cifar10], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

normalize = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean,std)])

train_data = datasets.CIFAR10(root= datapath, train=True, download = True, transform=normalize)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

valid_data = datasets.CIFAR10(root= datapath, train=False, download = True, transform=normalize)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=32, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


Convolution Neural Network Training and Validation

In [32]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = Net().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

epochs = 300
total_start_time = time.time()
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    losses = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses += loss.item()
        
    end_time = time.time()
    training_time = end_time - start_time
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {losses/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in valid_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()\

valid_accuracy = correct / total
print(f'Test Accuracy: {valid_accuracy * 100:.2f}%')

Epoch 1/300, Loss: 2.132779671836189, Training Time: 11.84 seconds
Epoch 11/300, Loss: 1.3054845870418268, Training Time: 11.77 seconds
Epoch 21/300, Loss: 1.0670512146623334, Training Time: 11.70 seconds
Epoch 31/300, Loss: 0.9023012640532666, Training Time: 11.75 seconds
Epoch 41/300, Loss: 0.7609419397108843, Training Time: 11.72 seconds
Epoch 51/300, Loss: 0.6202600797589437, Training Time: 11.77 seconds
Epoch 61/300, Loss: 0.47641500122296987, Training Time: 11.63 seconds
Epoch 71/300, Loss: 0.3387420295153149, Training Time: 11.69 seconds
Epoch 81/300, Loss: 0.2229321424087232, Training Time: 11.70 seconds
Epoch 91/300, Loss: 0.14075722430981052, Training Time: 11.65 seconds
Epoch 101/300, Loss: 0.08939099554341913, Training Time: 11.77 seconds
Epoch 111/300, Loss: 0.05943684597986483, Training Time: 11.72 seconds
Epoch 121/300, Loss: 0.042334682717490336, Training Time: 11.70 seconds
Epoch 131/300, Loss: 0.03189220657764016, Training Time: 11.71 seconds
Epoch 141/300, Loss: 0.02

Extended Convolution Neural Network Training and Validation

In [33]:
class ExtendedNet(nn.Module):
    def __init__(self):
        super(ExtendedNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)
        x = self.relu(self.conv3(x))
        x = self.maxpool(x)
        x = x.view(-1, 128 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Instantiate the model
model = ExtendedNet().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
epochs = 300
total_start_time = time.time()
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    losses = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses += loss.item()
        
    end_time = time.time()
    training_time = end_time - start_time
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {losses/len(train_loader)}, Training Time: {training_time:.2f} seconds')


total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in valid_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

valid_accuracy = correct / total
print(f'Test Accuracy: {valid_accuracy * 100:.2f}%')

Epoch 1/300, Loss: 2.2840374397758634, Training Time: 12.56 seconds
Epoch 11/300, Loss: 1.4350693683859177, Training Time: 11.54 seconds
Epoch 21/300, Loss: 1.1854603166619861, Training Time: 11.56 seconds
Epoch 31/300, Loss: 0.9991389659071914, Training Time: 11.56 seconds
Epoch 41/300, Loss: 0.8614900197764657, Training Time: 11.49 seconds
Epoch 51/300, Loss: 0.7440521800586678, Training Time: 11.44 seconds
Epoch 61/300, Loss: 0.6382345359293375, Training Time: 11.55 seconds
Epoch 71/300, Loss: 0.5386070913884858, Training Time: 11.53 seconds
Epoch 81/300, Loss: 0.44279941085127783, Training Time: 11.54 seconds
Epoch 91/300, Loss: 0.3498156610649897, Training Time: 11.55 seconds
Epoch 101/300, Loss: 0.2595750564376818, Training Time: 11.48 seconds
Epoch 111/300, Loss: 0.18173011208831386, Training Time: 11.53 seconds
Epoch 121/300, Loss: 0.116232008097654, Training Time: 11.53 seconds
Epoch 131/300, Loss: 0.06327274306221452, Training Time: 11.52 seconds
Epoch 141/300, Loss: 0.030406